In [48]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier

Etiquetamos los nombres de las características en el dataset.

In [49]:
featuresNames = ['MeanIntegratedProfile',
                'StdIntegratedProfile',
                'ExcessKurtosisIntegratedProfile',
                'SkewnessIntegratedProfile',
                'MeanDMSNRCurve',
                'StdDMSNRCurve',
                'ExcessKurtosisDMSNRCurve',
                'SkewnessDMSNRCurve',
                'Class']

Cargamos la informacion del dataset HTRU2.

In [50]:
data = pd.read_csv('../HTRU_2.csv',
                   header = None, 
                   names = featuresNames)

Separamos la variable categórica del dataset.

In [51]:
x = data.drop('Class', axis = 1, inplace = False)
y = data['Class']

Como podemos observar, existen variables que varian en un rango mayor de valores que otras. Para realizar una mejor comparacion entre las variables es importante estandarizar las variables con una media de cero y una desviación estándar de 1.

In [52]:
scaler = StandardScaler()
normalized_x = scaler.fit_transform(x)

Se divide el dataset en dos conjuntos de datos: entrenamiento y pruebas. El conjunto de entrenamiento estará compuesto por el 90% del dataset y pruebas del 10%.

El dataset HTRU2 es un conjunto de datos desbalanceado, por cada púlsar identificado, diez no lo son. Para que esta misma proporción se refleje en los conjuntos de entrenamiento y pruebas se realizará una división estratificada.

In [53]:
y.value_counts()

0    16259
1     1639
Name: Class, dtype: int64

In [54]:
x_train, x_test, y_train, y_test = train_test_split(normalized_x,
                                                    y,
                                                    test_size = 0.1,
                                                    random_state = 0,
                                                    stratify = y)

Ahora ambos conjuntos de datos tienen la misma proporción con respecto a la variable categórica.

In [55]:
print(y_train.value_counts())
print(y_test.value_counts())

0    14633
1     1475
Name: Class, dtype: int64
0    1626
1     164
Name: Class, dtype: int64


Como mencionamos anteriormente, el dataset HTRU2 es conjunto de datos desbalanceado. Para balancear utilizaremos una técnica de oversampling llamada SMOTE, donde se genera nuevas instancias de la clase minoritaria interpolando los valores de las instancias minoritarias más cercanas a una dada. El nuevo ratio será de 1:5, esto quiere decir, que por cada púlsar, cinco no lo serán en el conjunto de entrenamiento.

In [56]:
smote = SMOTE(random_state = 0,
             ratio = 0.2)
x_train_smote, y_train_smote = smote.fit_sample(x_train, y_train.ravel())

Ahora podemos observar que el conjunto de datos se encuentra balanceado.

In [57]:
np.bincount(y_train_smote)

array([14633,  2926])

Para encontrar los mejores hiper-parámetros que se ajusten al modelo utilizaremos la técnica de búsqueda en grilla. 

Para determinar los mejores valores de hidden layers, activation, solver y max_iter será necesario probar el modelo con diferentes valores.

En cada iteración se realizará validación cruzada con k = 5.

La métrica de evaluación será f1.

In [58]:
param_grid = {  
    'hidden_layer_sizes': [(100, ), (120, )],
    'activation': ['logistic', 'relu'],
    'solver': ['lbfgs', 'adam'],
    'max_iter': [200, 300, 500],
    'tol': [0.001]
}

grid_search = GridSearchCV(estimator = MLPClassifier(),
                           param_grid = param_grid,
                           scoring = 'f1',
                           cv = 5)

grid_search.fit(x_train_smote, y_train_smote)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_layer_sizes': [(100,), (120,)], 'activation': ['logistic', 'relu'], 'solver': ['lbfgs', 'adam'], 'max_iter': [200, 300, 500], 'tol': [0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [59]:
print('Tuneando los hiper-parámetros para f1')
print()

print('Los mejores hiper-parámetros encontrados con validación cruzada:')
print()
print(grid_search.best_params_)
print()
print('Puntajes de la métrica f1 en el conjunto de validación:')
print()
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print('%0.4f (+/-%0.04f) for %r'
          % (mean, std * 2, params))

Tuneando los hiper-parámetros para f1

Los mejores hiper-parámetros encontrados con validación cruzada:

{'activation': 'relu', 'hidden_layer_sizes': (120,), 'max_iter': 200, 'solver': 'lbfgs', 'tol': 0.001}

Puntajes de la métrica f1 en el conjunto de validación:

0.8973 (+/-0.0195) for {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'max_iter': 200, 'solver': 'lbfgs', 'tol': 0.001}
0.8964 (+/-0.0132) for {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'max_iter': 200, 'solver': 'adam', 'tol': 0.001}
0.8966 (+/-0.0092) for {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'max_iter': 300, 'solver': 'lbfgs', 'tol': 0.001}
0.8937 (+/-0.0151) for {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'max_iter': 300, 'solver': 'adam', 'tol': 0.001}
0.8997 (+/-0.0102) for {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'max_iter': 500, 'solver': 'lbfgs', 'tol': 0.001}
0.8950 (+/-0.0141) for {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'max_i

La búsqueda en grilla nos devuelve el modelo con los mejores hiper-parámetros.

Finalmente, vamos a predecir los resultados del modelo con los conjuntos de datos de entrenamiento y pruebas.

In [60]:
neural_net_model = grid_search

neural_net_model_train_prediction = neural_net_model.predict(x_train_smote)
neural_net_model_test_prediction = neural_net_model.predict(x_test)

In [61]:
print('Red neuronal')
print('\t-Mejores hiper-parámetros: ' + str(neural_net_model.best_params_))
print('\t-Puntaje f1 en entrenamiento: %.4f' % f1_score(y_train_smote, neural_net_model_train_prediction, average = 'binary'))
print('\t-Puntaje f1 en pruebas: %.4f' % f1_score(y_test, neural_net_model_test_prediction, average = 'binary'))

Red neuronal
	-Mejores hiper-parámetros: {'activation': 'relu', 'hidden_layer_sizes': (120,), 'max_iter': 200, 'solver': 'lbfgs', 'tol': 0.001}
	-Puntaje f1 en entrenamiento: 0.9151
	-Puntaje f1 en pruebas: 0.9024
